In [1]:
pip install bs4

  Created wheel for bs4: filename=bs4-0.0.1-py3-none-any.whl size=1273 sha256=f64e2621ebb15b80970cbbd2af6617a9025329a98fec84574d4ca8a67627de74
  Stored in directory: c:\users\alecl\appdata\local\pip\cache\wheels\0a\9e\ba\20e5bbc1afef3a491f0b3bb74d508f99403aabe76eda2167ca
Successfully built bs4
Note: you may need to restart the kernel to use updated packages.


In [2]:
conda install -c conda-forge parse

Solving environment: ...working... done

## Package Plan ##

  environment location: C:\Users\alecl\Anaconda3

  added / updated specs:
    - parse


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    certifi-2020.12.5          |   py37h03978a9_1         143 KB  conda-forge
    parse-1.18.0               |     pyh9f0ad1d_0          21 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         165 KB

The following NEW packages will be INSTALLED:

  parse              conda-forge/noarch::parse-1.18.0-pyh9f0ad1d_0

The following packages will be UPDATED:

  certifi            pkgs/main::certifi-2020.12.5-py37haa9~ --> conda-forge::certifi-2020.12.5-py37h03978a9_1

The following packages will be SUPERSEDED by a higher-priority channel:

  ca-certificates    pkgs/main::ca-certificates-2020.12.8-~ --> conda-forge::ca

In [3]:
pip install lxml

Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install folium

Note: you may need to restart the kernel to use updated packages.


# Part 1: Importing Table

In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
print("Imported!")
import lxml.html as lh
import folium

Imported!


In [2]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

In [3]:
extract_data = requests.get(url).text
data = BeautifulSoup(extract_data, 'lxml')

In [4]:
cols = ['Postalcode','Borough','Neighborhood']
TO = pd.DataFrame(columns = cols)
content = data.find('div', class_='mw-parser-output')
table = content.table.tbody
Postalcode = 0
Borough = 0
Neighborhood = 0

In [5]:
# Parsing through the data

for tr in table.find_all('tr'):
    i = 0
    for td in tr.find_all('td'):
        if i == 0:
            Postalcode = td.text
            i = i + 1
        elif i == 1:
            Borough = td.text
            i = i + 1
        elif i == 2: 
            Neighborhood = td.text.strip('\n').replace(']','')
    TO = TO.append({'Postalcode': Postalcode,'Borough': Borough,'Neighborhood': Neighborhood},ignore_index=True)

In [6]:
TO = TO[TO.Borough!='Not assigned']
TO = TO[TO.Borough!= 0]
TO.reset_index(drop = True, inplace = True)
i = 0


In [7]:
# Now to create the loop for 
for i in range(0,TO.shape[0]):
    if TO.iloc[i][2] == 'Not assigned':
        TO.iloc[i][2] = TO.iloc[i][1]
        i = i+1
        



In [8]:
df = TO.groupby(['Postalcode','Borough'])['Neighborhood'].apply(', '.join).reset_index()

In [9]:
# Dropping Nan values and Not Assigned values from the dataframe
df = df.dropna()
df = df[~df.Borough.str.contains("Not assigned")]

In [10]:
def neighborhood(grouped):    
    return ', '.join(sorted(grouped['Neighborhood'].tolist()))
                    
grp = df.groupby(['Postalcode', 'Borough'])
df_final = grp.apply(neighborhood).reset_index(name='Neighborhood')

In [11]:
print(df_final.shape)

(103, 3)


In [12]:
df_final

,Postalcode,Borough,Neighborhood
0,M1B\n,Scarborough\n,"Malvern, Rouge"
1,M1C\n,Scarborough\n,"Rouge Hill, Port Union, Highland Creek"
2,M1E\n,Scarborough\n,"Guildwood, Morningside, West Hill"
3,M1G\n,Scarborough\n,Woburn
4,M1H\n,Scarborough\n,Cedarbrae
...,...,...,...
98,M9N\n,York\n,Weston
99,M9P\n,Etobicoke\n,Westmount
100,M9R\n,Etobicoke\n,"Kingsview Village, St. Phillips, Martin Grove ..."
101,M9V\n,Etobicoke\n,"South Steeles, Silverstone, Humbergate, Jamest..."


In [13]:
# Removing \n from the string values in the Postalcode column
df_final.Postalcode = df_final.Postalcode.str.replace('\n','')

# Part 2: Geographic Coordinates

In [14]:
# Importing data
df_geo = pd.read_csv(r'C:\Users\alecl\Desktop\Certificates\Coursera\Geo.csv')

In [15]:
df_geo.rename(columns={'Postal Code':'Postalcode'}, inplace=True)

In [16]:
df_merged = pd.merge(df_final, df_geo, on='Postalcode')

In [18]:
df_merged.shape

(103, 5)

In [17]:
df_merged

,Postalcode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough\n,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough\n,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough\n,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough\n,Woburn,43.770992,-79.216917
4,M1H,Scarborough\n,Cedarbrae,43.773136,-79.239476
...,...,...,...,...,...
98,M9N,York\n,Weston,43.706876,-79.518188
99,M9P,Etobicoke\n,Westmount,43.696319,-79.532242
100,M9R,Etobicoke\n,"Kingsview Village, St. Phillips, Martin Grove ...",43.688905,-79.554724
101,M9V,Etobicoke\n,"South Steeles, Silverstone, Humbergate, Jamest...",43.739416,-79.588437
